In [1]:
import pandas as pd
import xml.etree.ElementTree as et
import numpy as np
import sqlite3
pd.options.mode.chained_assignment = None

In [2]:
conn = sqlite3.connect("../data/eusoccerdatabase.sqlite")
query = "SELECT id, country_id, league_id, season, date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, B365H, B365D, B365A, shoton, shotoff FROM Match"

df = pd.read_sql_query(query, conn)
df_shots = df.dropna(subset=['shoton', 'shotoff'])
df_shots.set_index('id')

,country_id,league_id,season,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,B365A,shoton,shotoff
id,,,,,,,,,,,,,,
1729,1729,1729,2008/2009,2008-08-17 00:00:00,489042,10260,10261,1,1,1.29,5.5,11.00,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...
1730,1729,1729,2008/2009,2008-08-16 00:00:00,489043,9825,8659,1,0,1.20,6.5,15.00,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...
1731,1729,1729,2008/2009,2008-08-16 00:00:00,489044,8472,8650,0,1,5.50,3.6,1.67,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...
1732,1729,1729,2008/2009,2008-08-16 00:00:00,489045,8654,8528,2,1,1.91,3.4,4.20,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...
1733,1729,1729,2008/2009,2008-08-17 00:00:00,489046,10252,8456,4,2,1.91,3.4,4.33,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25945,24558,24558,2015/2016,2016-05-25 00:00:00,1992225,9931,9956,0,1,NaN,NaN,NaN,<shoton />,<shotoff />
25946,24558,24558,2015/2016,2016-05-25 00:00:00,1992226,7896,10190,3,0,NaN,NaN,NaN,<shoton />,<shotoff />
25947,24558,24558,2015/2016,2016-05-25 00:00:00,1992227,10199,10179,2,2,NaN,NaN,NaN,<shoton />,<shotoff />


In [3]:
home_shoton = []
home_shotoff = []
home_shots = []
home_shots_acc = []

away_shoton = []
away_shotoff = []
away_shots = []
away_shost_acc = []


for index, row in df_shots.iterrows():
    home_id = row['home_team_api_id']
    away_id = row['away_team_api_id']
    home_shoton_counter = 0
    home_shotoff_counter = 0
    home_shot_accuracy = 0.0
    home_shots_total = 0
    away_shoton_counter = 0
    away_shotoff_counter = 0
    away_shot_accuracy = 0.0
    away_shots_total = 0
    shoton_root = et.fromstring(str(row['shoton']))
    shotoff_root = et.fromstring(str(row['shotoff']))
    for val in shoton_root.findall('value'):
        if val.find('team') is not None:
            team_id = int(val.find('team').text)
            if team_id == home_id:
                home_shoton_counter = home_shoton_counter + 1
                home_shots_total = home_shots_total + 1
            elif team_id == away_id:
                away_shoton_counter = away_shoton_counter + 1
                away_shots_total = away_shots_total + 1
        
        
    for val2 in shotoff_root.findall('value'):
        if val2.find('team') is not None:
            team_id = int(val2.find('team').text)
            if team_id == home_id:
                home_shotoff_counter = home_shotoff_counter + 1
                home_shots_total = home_shots_total + 1
            elif team_id == away_id:
                away_shotoff_counter = away_shotoff_counter + 1
                away_shots_total = away_shots_total + 1
    
    if home_shots_total != 0:
        home_shot_accuracy = home_shoton_counter / (home_shoton_counter + home_shotoff_counter)
        
    if away_shots_total != 0:
        away_shot_accuracy = away_shoton_counter / (away_shoton_counter + away_shotoff_counter)
    
#    print('Home_Team_ID: ' + str(home_id) + ' Shoton: ' + str(home_shoton_counter) + ' Shotoff: ' + str(home_shotoff_counter) + ' Shot Accuracy: ' + str(home_shot_accuracy))
#    print('Away_Team_ID: ' + str(away_id) + ' Shoton: ' + str(away_shoton_counter) + ' Shotoff: ' + str(away_shotoff_counter) + ' Shot Accuracy: ' + str(away_shot_accuracy))
 
    home_shoton.append(home_shoton_counter)
    home_shotoff.append(home_shotoff_counter)
    home_shots.append(home_shots_total)
    home_shots_acc.append(home_shot_accuracy)
    
    away_shoton.append(away_shoton_counter)
    away_shotoff.append(away_shotoff_counter)
    away_shots.append(away_shots_total)
    away_shost_acc.append(away_shot_accuracy)

df_shots['home_team_shots'] = home_shots
df_shots['home_team_shots_on_target'] = home_shoton
df_shots['home_team_shots_off_target'] = home_shotoff
df_shots['home_team_shot_accuracy'] = home_shots_acc
    
df_shots['away_team_shots'] = away_shots
df_shots['away_team_shots_on_target'] = away_shoton
df_shots['away_team_shots_off_target'] = away_shotoff
df_shots['away_team_shot_accuracy'] = away_shost_acc
    

In [4]:
df_shots

,id,country_id,league_id,season,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,shoton,shotoff,home_team_shots,home_team_shots_on_target,home_team_shots_off_target,home_team_shot_accuracy,away_team_shots,away_team_shots_on_target,away_team_shots_off_target,away_team_shot_accuracy
1728,1729,1729,1729,2008/2009,2008-08-17 00:00:00,489042,10260,10261,1,1,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,21,11,10,0.523810,10,1,9,0.100000
1729,1730,1729,1729,2008/2009,2008-08-16 00:00:00,489043,9825,8659,1,0,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,25,12,13,0.480000,5,2,3,0.400000
1730,1731,1729,1729,2008/2009,2008-08-16 00:00:00,489044,8472,8650,0,1,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,7,4,3,0.571429,16,11,5,0.687500
1731,1732,1729,1729,2008/2009,2008-08-16 00:00:00,489045,8654,8528,2,1,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,12,5,7,0.416667,22,7,15,0.318182
1732,1733,1729,1729,2008/2009,2008-08-17 00:00:00,489046,10252,8456,4,2,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,9,5,4,0.555556,14,9,5,0.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25944,25945,24558,24558,2015/2016,2016-05-25 00:00:00,1992225,9931,9956,0,1,...,<shoton />,<shotoff />,0,0,0,0.000000,0,0,0,0.000000
25945,25946,24558,24558,2015/2016,2016-05-25 00:00:00,1992226,7896,10190,3,0,...,<shoton />,<shotoff />,0,0,0,0.000000,0,0,0,0.000000
25946,25947,24558,24558,2015/2016,2016-05-25 00:00:00,1992227,10199,10179,2,2,...,<shoton />,<shotoff />,0,0,0,0.000000,0,0,0,0.000000
25947,25948,24558,24558,2015/2016,2016-05-25 00:00:00,1992228,10191,10192,0,3,...,<shoton />,<shotoff />,0,0,0,0.000000,0,0,0,0.000000


In [5]:
df_shots=df_shots[df_shots['home_team_shot_accuracy']!=0].dropna()
df_shots=df_shots[df_shots['B365A']!=0].dropna()
df_shots=df_shots[df_shots['B365H']!=0].dropna()
df_shots=df_shots[df_shots['B365D']!=0].dropna()
df_csv=df_shots.drop(columns=['shoton','shotoff'],axis=1)
df_csv

,id,country_id,league_id,season,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,...,B365D,B365A,home_team_shots,home_team_shots_on_target,home_team_shots_off_target,home_team_shot_accuracy,away_team_shots,away_team_shots_on_target,away_team_shots_off_target,away_team_shot_accuracy
1728,1729,1729,1729,2008/2009,2008-08-17 00:00:00,489042,10260,10261,1,1,...,5.50,11.00,21,11,10,0.523810,10,1,9,0.100000
1729,1730,1729,1729,2008/2009,2008-08-16 00:00:00,489043,9825,8659,1,0,...,6.50,15.00,25,12,13,0.480000,5,2,3,0.400000
1730,1731,1729,1729,2008/2009,2008-08-16 00:00:00,489044,8472,8650,0,1,...,3.60,1.67,7,4,3,0.571429,16,11,5,0.687500
1731,1732,1729,1729,2008/2009,2008-08-16 00:00:00,489045,8654,8528,2,1,...,3.40,4.20,12,5,7,0.416667,22,7,15,0.318182
1732,1733,1729,1729,2008/2009,2008-08-17 00:00:00,489046,10252,8456,4,2,...,3.40,4.33,9,5,4,0.555556,14,9,5,0.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24552,24553,21518,21518,2015/2016,2015-10-25 00:00:00,2030167,9906,10267,2,1,...,3.80,6.50,14,9,5,0.642857,6,2,4,0.333333
24553,24554,21518,21518,2015/2016,2015-10-24 00:00:00,2030168,9864,9783,2,0,...,3.25,3.40,6,3,3,0.500000,9,4,5,0.444444
24554,24555,21518,21518,2015/2016,2015-10-26 00:00:00,2030169,8315,9869,3,0,...,4.00,7.00,15,3,12,0.200000,6,2,4,0.333333
24555,24556,21518,21518,2015/2016,2015-10-24 00:00:00,2030170,7878,8603,1,1,...,3.25,3.25,13,7,6,0.538462,3,3,0,1.000000


In [6]:
df_csv.to_csv('../data/matches_all_with_shots.csv')